In [1]:
!pip3 install torch==1.10.2+cu113 torchvision==0.11.3+cu113 torchaudio==0.10.2+cu113 -f https://download.pytorch.org/whl/cu113/torch_stable.html

Looking in links: https://download.pytorch.org/whl/cu113/torch_stable.html
     |██████████████▋                 | 834.1 MB 1.2 MB/s eta 0:14:04tcmalloc: large alloc 1147494400 bytes == 0x559ebf3ba000 @  0x7fc57ee4d615 0x559e86c603bc 0x559e86d4118a 0x559e86c631cd 0x559e86d55b3d 0x559e86cd7458 0x559e86cd202f 0x559e86c64aba 0x559e86cd72c0 0x559e86cd202f 0x559e86c64aba 0x559e86cd3cd4 0x559e86d56986 0x559e86cd3350 0x559e86d56986 0x559e86cd3350 0x559e86d56986 0x559e86cd3350 0x559e86c64f19 0x559e86ca8a79 0x559e86c63b32 0x559e86cd71dd 0x559e86cd202f 0x559e86c64aba 0x559e86cd3cd4 0x559e86cd202f 0x559e86c64aba 0x559e86cd2eae 0x559e86c649da 0x559e86cd3108 0x559e86cd202f
     |██████████████████▌             | 1055.7 MB 1.2 MB/s eta 0:10:46tcmalloc: large alloc 1434370048 bytes == 0x559f03a10000 @  0x7fc57ee4d615 0x559e86c603bc 0x559e86d4118a 0x559e86c631cd 0x559e86d55b3d 0x559e86cd7458 0x559e86cd202f 0x559e86c64aba 0x559e86cd72c0 0x559e86cd202f 0x559e86c64aba 0x559e86cd3cd4 0x559e86d56986 0x559e

In [2]:
!pip3 install transformers
!pip3 install hugsvision

     |████████████████████████████████| 3.5 MB 5.3 MB/s 
     |████████████████████████████████| 6.8 MB 75.0 MB/s 
     |████████████████████████████████| 596 kB 75.7 MB/s 
     |████████████████████████████████| 67 kB 5.9 MB/s 
     |████████████████████████████████| 895 kB 81.6 MB/s 
  Attempting uninstall: pyyaml
    Found existing installation: PyYAML 3.13
    Uninstalling PyYAML-3.13:
      Successfully uninstalled PyYAML-3.13
     |████████████████████████████████| 431 kB 7.8 MB/s 
     |████████████████████████████████| 527 kB 75.3 MB/s 
     |████████████████████████████████| 397 kB 84.3 MB/s 
     |████████████████████████████████| 952 kB 75.2 MB/s 
     |████████████████████████████████| 829 kB 73.1 MB/s 
     |████████████████████████████████| 133 kB 73.6 MB/s 
     |████████████████████████████████| 1.1 MB 70.5 MB/s 
     |████████████████████████████████| 271 kB 75.8 MB/s 
     |████████████████████████████████| 144 kB 81.3 MB/s 
     |████████████████████████████████| 94 

In [4]:
from google.colab import drive
drive.mount('/content/drive')

Drive already mounted at /content/drive; to attempt to forcibly remount, call drive.mount("/content/drive", force_remount=True).


In [ ]:
!cd XRay/
!ls -a
!rm -r XRay/.ipynb_checkpoints/

In [ ]:
import transformers
from transformers import AutoFeatureExtractor, DeiTForImageClassification ,DeiTFeatureExtractor , ViTForImageClassification , ViTFeatureExtractor , ViTModel
from PIL import Image
import requests
import torch
import argparse
from torchvision import datasets
from torchvision import transforms as tfc
from torchmetrics import Accuracy
from torch.utils.data import DataLoader
import tqdm
import os
import numpy as np
import math
import pytorch_lightning as pl
import matplotlib.pyplot as plt
import matplotlib.image as mpimg
import requests
try:
    import hugsvision
except:
    !pip install -q hugsvision
    import hugsvision
    
print(hugsvision.__version__)

from hugsvision.inference.VisionClassifierInference import VisionClassifierInference
from hugsvision.nnet.VisionClassifierTrainer import VisionClassifierTrainer
from hugsvision.dataio.VisionDataset import VisionDataset
from google.colab import drive

from PIL import ImageFile
ImageFile.LOAD_TRUNCATED_IMAGES = True

dataset_dir = "./XRay/"
ratio = 0.2
lr, num_epochs = 2e-5, 100
save_dir = "./checkpoint/"
no_cuda = False
projectname = "XRayVitExp"
model_pretrained = "google/vit-base-patch16-224-in21k"
# def train(num_epochs,no_cuda,save_dir, projectname, batch_size,dataset_dir,model_name, ratio = 0.2, lr = 2e-5):
    # device = torch.device('cuda' if torch.cuda.is_available() and not no_cuda else 'cpu')



train_ds, test_ds, id2label, label2id = VisionDataset.fromImageFolder( 
    dataset_dir 
    ,test_ratio=ratio
    ,balanced=True
    ,augmentation=False
    ,torch_vision=True
)



/usr/local/lib/python3.7/dist-packages/torchvision/transforms/transforms.py:288: UserWarning: Argument interpolation should be of type InterpolationMode instead of int. Please, use InterpolationMode enum.
  "Argument interpolation should be of type InterpolationMode instead of int. "
/usr/local/lib/python3.7/dist-packages/torchvision/transforms/transforms.py:288: UserWarning: Argument interpolation should be of type InterpolationMode instead of int. Please, use InterpolationMode enum.
  "Argument interpolation should be of type InterpolationMode instead of int. "


0.75.3
Split Datasets...
Balance train dataset...
The less represented label in train as 2008 occurrences
Size of train after balancing is 4016
train_ds:  3212


In [2]:
model_name = "Vit"
batch_size = 512
if model_name == 'Deit':
    model_pretrained = "facebook/deit-base-distilled-patch16-224"
    trainer = VisionClassifierTrainer(
        model_name=projectname,
        train=train_ds,
        test=test_ds,
        output_dir=save_dir,
        max_epochs=num_epochs,
        batch_size=batch_size,  # On RTX 2080 Ti
        lr	= lr,
        fp16	     = False,
        model = DeiTForImageClassification.from_pretrained(
            model_pretrained,
            num_labels = len(label2id),
            label2id   = label2id,
            id2label   = id2label
        ),
        feature_extractor = DeiTFeatureExtractor.from_pretrained(
            model_pretrained,
        ),
    )
else:
    trainer = VisionClassifierTrainer(
        model_name=projectname,
        train=train_ds,
        test=test_ds,
        output_dir=save_dir,
        max_epochs=num_epochs,
        batch_size=batch_size,  # On RTX 2080 Ti
        lr	= lr,
        # fp16	     = False,
        model = ViTForImageClassification.from_pretrained(
            model_pretrained,
            num_labels = len(label2id),
            label2id   = label2id,
            id2label   = id2label
        ),
        feature_extractor = ViTFeatureExtractor.from_pretrained(
            model_pretrained,
        ),
    )

NameError: ignored